<a href="https://colab.research.google.com/github/NikolasGialitsis/euCovid-travel-model/blob/master/EU_Covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/NikolasGialitsis/euCovid-travel-model

Cloning into 'euCovid-travel-model'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 87 (delta 15), reused 69 (delta 7), pack-reused 0
Unpacking objects: 100% (87/87), done.


In [11]:
!pip install python-copasi
%cd /content/euCovid-travel-model/generating

/content/euCovid-travel-model/generating


In [9]:
!mv /content/euCovid-travel-model/EU_expand_model.py /content/euCovid-travel-model/generating

In [14]:
!python /content/euCovid-travel-model/generating/EU_expand_model.py /content/euCovid-travel-model/generating/SEIR_base.cps eu_usa.cps

abrev: {'GR': 0, 'IT': 1, 'BU': 2}
tcmalloc: large alloc 1449377792 bytes == 0x1e54000 @  0x7f75c2b22001 0x582743 0x582af3 0x50d164 0x507d64 0x509a90 0x50a48d 0x50bfb4 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00 0x7f75c271db97 0x5b250a
compartment name =  COVID_GR
state_abrev =  GR
GR 0
state flux =  [{0: 0.1, 1: 0.005, 2: 0.01}, {0: 0.1, 1: 0.0001, 2: 0.002}]
compartment name =  COVID_IT
state_abrev =  IT
IT 1
state flux =  [{0: 0.0001, 1: 0.02, 2: 0.0025}, {0: 0.005, 1: 0.02, 2: 0.0005}]
compartment name =  COVID_BU
state_abrev =  BU
BU 2
state flux =  [{0: 0.002, 1: 0.0005, 2: 0.006666666666666667}, {0: 0.01, 1: 0.0025, 2: 0.006666666666666667}]
Number of assigments = 12

Number of assigments = 12



In [15]:
%cd /content/
!kaggle datasets download -d shahules/european-countries-population

/content
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [53]:
%matplotlib inline
import matplotlib.pyplot as plt
import sys
import os
import numpy as np
import pandas as pd
from sklearn import datasets, linear_model
from random import sample 
from google.colab import drive

def bash_collect_data():
  sys.path.append('/usr/local/lib/python3.7/site-packages/')
  drive.mount('/content/drive')
  os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/My Drive/Colab Notebooks/.kaggle/"
  !kaggle datasets download -d sudalairajkumar/novel-corona-virus-2019-dataset
  !kaggle datasets download -d duvallwh/us-asia-and-europe-population-and-area-data
  !unzip /content/european-countries-population.zip
  !unzip /content/novel-corona-virus-2019-dataset.zip
  !unzip /content/us-asia-and-europe-population-and-area-data.zip


def PopsAreas2dict(fn='/content/area_pop_data.csv'):
    df = pd.read_csv(fn)
    df = df[df['region'] == 'Europe']
    print(df.columns)
    areas = {}
    pops = {}
    for row in df.iterrows():
      name = row[1]['country'].upper()
      name = name.split(' ')[0]
      area = row[1]['area_km']
      pop = row[1]['population']
      if name not in areas.keys():
        areas[name] = area
        pops[name] = pop
      else:
        areas[name] += area 
        pops[name] += pop
    return pops,areas

def CovidCountry(fn,countries):
  df = pd.read_csv(fn)
  df = df.drop(labels=['Lat','Long'],axis='columns')
  df['lastmonth'] = df[df.columns[-30:]].sum(axis=1) 
  infected = {}
  for row in df.iterrows():
    name = row[1]['Country/Region'].upper()
    name = name.split(' ')[0]
    confirmed = row[1]['lastmonth']
    if name not in countries:
      continue
    if name not in infected.keys():
      infected[name] = confirmed
    else:
      infected[name] += confirmed
  return infected

def CountryTravel(fn):
  


#bash_collect_data()

pops,areas =  PopsAreas2dict('/content/area_pop_data.csv')
#print(pops)
EUcountries = pops.keys()
print(pops.keys())
infected = CovidCountry(fn='/content/time_series_covid_19_confirmed.csv',countries=EUcountries)


Index(['Unnamed: 0', 'region', 'country', 'province', 'area_km', 'population'], dtype='object')
dict_keys(['ALBANIA', 'ANDORRA', 'ARMENIA', 'AUSTRIA', 'AZERBAIJAN', 'BELARUS', 'BELGIUM', 'BOSNIA', 'BULGARIA', 'CROATIA', 'CYPRUS', 'CZECH', 'DENMARK', 'ESTONIA', 'FAROE', 'FINLAND', 'FRANCE', 'GEORGIA', 'GERMANY', 'GIBRALTAR', 'GREECE', 'GUERNSEY', 'HUNGARY', 'ICELAND', 'REPUBLIC', 'ISLE', 'ITALY', 'JERSEY', 'KAZAKHSTAN', 'KOSOVO', 'LATVIA', 'LIECHTENSTEIN', 'LITHUANIA', 'LUXEMBOURG', 'MALTA', 'MOLDOVA', 'MONACO', 'MONTENEGRO', 'NETHERLANDS', 'NORTH', 'NORWAY', 'POLAND', 'PORTUGAL', 'ROMANIA', 'RUSSIA', 'SAN', 'SERBIA', 'SLOVAKIA', 'SLOVENIA', 'SPAIN', 'MAYEN', 'SWEDEN', 'SWITZERLAND', 'TRANSNISTRIA', 'TURKEY', 'UKRAINE', 'UNITED', 'VATICAN', 'ÅLAND'])
0      717859
1       48138
2      329002
3       25329
4        4270
        ...  
261     18400
262     21611
263      5598
264    148233
265       222
Name: lastmonth, Length: 266, dtype: int64
dict_keys(['ALBANIA', 'ANDORRA', 'ARMENIA',